## Problem 1: Geocode shopping centers (5 points)

The overall aim of problems 1-3 is to find out **how many people live within a walking distance (1.5 km) from certain shopping centres in Helsinki**.

Problem 1 concerns the locations of shopping centres: find their addresses and translate them into coordinates.

---

### a) Prepare an input file containing the addresses of shopping centres

Find out the addresses of the following shopping centres (e.g., by using your favourite search engine), and collect them in a text file called `shopping_centres.txt`:

 - Itis
 - Forum
 - Iso-omena
 - Sello
 - Jumbo
 - REDI
 - Tripla 
 
The text file should be in semicolon-separated format (`;`) and include the following columns:

- `id` (integer) a unique identifier for each shopping centre (a
- `name` (string) of each shopping center
- `addr` (string) the address 


See an example of how to format the text file [in the lesson 3 materials](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/geocoding-in-geopandas.html). Remember to *add*, *commit*, and *push* the file to your git repository.

---


### a) Prepare an input file containing the addresses of shopping centres

I decided to find out the addresses of the following shopping centres located within Krakow (e.g., by using your favourite search engine), and collect them in a text file called `shopping_centres.txt`:

 - Galeria Krakowska
 - Centrum Handlowe Bonarka
 - Centrum Siewna
 - Bronowice Mall
 - Shopping Centre Krokus
 - M1 Krakow
 - Jubilat
 
The text file should be in semicolon-separated format (`;`) and include the following columns:

- `id` (integer) a unique identifier for each shopping centre (a
- `name` (string) of each shopping center
- `addr` (string) the address 


See an example of how to format the text file [in the lesson 3 materials](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/geocoding-in-geopandas.html). Remember to *add*, *commit*, and *push* the file to your git repository.

---

### b) Read the list of addresses

Read the list of addresses you just prepared into a `pandas.DataFrame` called `shopping_centres`

In [1]:
# ADD YOUR OWN CODE HERE
import pandas as pd

import pathlib 
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"

shopping_centres = pd.read_csv(DATA_DIRECTORY / "shopping_centres.txt", sep=";")

FileNotFoundError: [Errno 2] No such file or directory: '/home/stephen137/GIS/exercises/data/shopping_centres.txt'

In [17]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
import pandas
assert isinstance(shopping_centres, pandas.DataFrame)
for column in ("id", "name", "addr"):
    assert column in shopping_centres.columns


---

### c) Geocode the addresses

Geocode the addresses using the Nominatim geocoding service. Join the results with the input data, and store them in a `geopandas.GeoDataFrame` with the same name (`shopping_centres`). 

Remember to define a custom `user_agent` string!

In [18]:
# ADD YOUR OWN CODE HERE

# Geocode the addresses using the Nominatim geocoding service

import geopandas as gpd

geocoded_addresses = gpd.tools.geocode(
    shopping_centres["addr"],
    provider="nominatim",
    user_agent="autogis2022",
    timeout=10
)
geocoded_addresses

,geometry,address
0,POINT (19.94615 50.06759),"Galeria Krakowska, 5, Pawia, Wesoła, Stare Mia..."
1,POINT (19.95072 50.02798),"Bonarka City Center, 11, Generała Henryka Kami..."
2,POINT (19.94823 50.09634),"Biedronka, 46, Siewna, Prądnik Biały, Kraków, ..."
3,POINT (19.89828 50.09209),"Galeria Bronowice, 61, Stawowa, Bronowice Wiel..."
4,POINT (19.93062 50.01652),"Cinema City, 62, Zakopiańska, Borek Fałęcki, Ł..."
5,POINT (19.99900 50.06407),"mBank, 67, Aleja Pokoju, Beszcz, Łęg, Czyżyny,..."
6,POINT (19.92761 50.05572),"1, Aleja Zygmunta Krasińskiego, Zwierzyniec, K..."


In [20]:
# Join the results with the input data, and store them in a geopandas.GeoDataFrame with the same name (shopping_centres)

shopping_centres = geocoded_addresses.join(shopping_centres)

In [21]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
import geopandas
assert isinstance(shopping_centres, geopandas.GeoDataFrame)
for column in ("id", "name", "addr", "geometry"):
    assert column in shopping_centres.columns

Check that the coordinate reference system of the geocoded result is correctly defined, and **reproject the layer into ETRS GK-25** (EPSG:3879):

In [22]:
# ADD YOUR OWN CODE HERE
shopping_centres.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [23]:
# Transform the data from WGS84 to an EPSG:3879 projection.
shopping_centres = shopping_centres.to_crs("EPSG:3879")

In [24]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
import pyproj
assert shopping_centres.crs == pyproj.CRS("EPSG:3879")


---

### d) Save the resulting vector data set

Save `shopping_centres` as a *GeoPackage* named `shopping_centres.gpkg`:

In [25]:
# ADD YOUR OWN CODE HERE

shopping_centres.to_file(DATA_DIRECTORY / "shopping_centres.gpkg")

In [26]:
shopping_centres

,geometry,address,id,name,addr
0,POINT (25138251.060 5560613.785),"Galeria Krakowska, 5, Pawia, Wesoła, Stare Mia...",1,Galeria Krakowska,"Pawia 5, 31-154 Kraków, Poland"
1,POINT (25138280.056 5556188.048),"Bonarka City Center, 11, Generała Henryka Kami...",2,Centrum Handlowe Bonarka,"Henryka Kamieńskiego 11, 30-644 Kraków, Poland"
2,POINT (25138616.805 5563799.309),"Biedronka, 46, Siewna, Prądnik Biały, Kraków, ...",3,Centrum Siewna,"Siewna 46, 31-231 Kraków, Poland"
3,POINT (25135013.131 5563569.613),"Galeria Bronowice, 61, Stawowa, Bronowice Wiel...",4,Bronowice Mall,"Stawowa 61, 31-346 Kraków, Poland"
4,POINT (25136754.156 5555011.892),"Cinema City, 62, Zakopiańska, Borek Fałęcki, Ł...",5,Park Handlowy Zakopianka,"Zakopiańska 62, 30-418 Kraków, Poland"
5,POINT (25142006.343 5559966.534),"mBank, 67, Aleja Pokoju, Beszcz, Łęg, Czyżyny,...",6,M1 Krakow,"Aleja Pokoju 67, 31-580 Kraków, Poland"
6,POINT (25136834.711 5559383.795),"1, Aleja Zygmunta Krasińskiego, Zwierzyniec, K...",7,Jubilat,"Aleja Zygmunta Krasińskiego 1, 31-111 Kraków, ..."



---

Well done! Now you can continue to [problem 2](Exercise-3-Problem-2.ipynb)